 #  RDD API Examples

In [1]:
%%spark

## Word Count
In this example, we use a few transformations to build a dataset of (String, Int) pairs called counts and then save it to a file.
```
sc.textFile(name, minPartitions=None, use_unicode=True)
Read a text file from HDFS, a local file system (available on all nodes), or any Hadoop-supported file system URI, and return it as an RDD of Strings.
```

In [2]:
val textFile = sc.textFile("../datasets/quijote.txt")
val counts = textFile.flatMap(line => line.split(" "))
                 .map(word => (word, 1))
                 .reduceByKey(_ + _)
counts.sortBy(_._2, ascending=false).take(50)



[(que,19429), (de,17985), (y,15894), (la,10200), (a,9519), (,9195), (el,7957), (en,7898), (no,5603), (se,4690), (los,4680), (con,4047), (por,3758), (las,3423), (lo,3387), (le,3382), (su,3319), (don,2533), (del,2464), (me,2344), (como,2226), (es,1990), (un,1927), (más,1823), (si,1779), (yo,1703), (al,1696), (mi,1684), (para,1419), (ni,1350), (una,1300), (y,,1250), (tan,1217), (porque,1189), (o,1159), (sin,1139), (que,,1069), (sus,1047), (ha,1038), (él,1034), (había,1006), (ser,997), (todo,963), (Sancho,950), (Quijote,893), (-dijo,873), (bien,862), (-respondió,813), (vuestra,792), (señor,732)]

 ## Pi Estimation

Spark can also be used for compute-intensive tasks. This code estimates π by "throwing darts" at a circle. We pick random points in the unit square ((0, 0) to (1,1)) and see how many fall in the unit circle. The fraction should be π / 4, so we use this to get our estimate.

In [5]:
val NUM_SAMPLES = 1000000 
val count = sc.parallelize(1 to NUM_SAMPLES).filter { _ =>
  val x = math.random
  val y = math.random
  x*x + y*y < 1
}.count()
println(s"Pi is roughly ${4.0 * count / NUM_SAMPLES}")

Pi is roughly 3.14222


null

# DataFrame API Examples

In this example, we count all quijote lines mentioning Dulcinea.

In [8]:
import org.apache.spark.sql.functions.col

val textFile = sc.textFile("../datasets/quijote.txt")

// Creates a DataFrame having a single column named "line"
val df = textFile.toDF("line")
val dulcinea_lines = df.filter(col("line").like("%Dulcinea%"))
// Counts all the Dulcinea lines
println(dulcinea_lines.count())
// Counts lines mentioning Dulcinea and Quijote
val dulcinea_quijote_lines = dulcinea_lines.filter(col("line").like("%Quijote%"))
println(dulcinea_quijote_lines.count())
// Fetches the lines as an array of strings
dulcinea_quijote_lines

282
12


In [16]:
dulcinea_quijote_lines.display(1000)